In [36]:
# zip 파일을 읽기 위한 라이브러리 가져오기
import zipfile

# json 파일을 읽기 위한 라이브러리 가져오기
import json

#
from tqdm import tqdm
import pickle
import pandas as pd

In [37]:
zipfilename = "../NIKL_OM_2021_v1.1.zip"

# 발화 리스트 초기화
utterances_list = [] 

# 발화자 리스트 초기화
speakers_list = []

# 압축된 zip 파일 읽기
with zipfile.ZipFile(zipfilename) as z:
  # 말뭉치 압축 파일(z) 내에 들어 있는 파일을 하나씩 처리하기
  for filename in tqdm(z.namelist()):
    # 파일이 JSON 이외의 형식인 경우 읽지 않고 넘어가기
    if not filename.endswith(('.json', '.JSON')):
      continue

    # JSON 파일을 열어서 그 내용을 data라는 이름으로 저장하기
    with z.open(filename) as f:  
      data = json.loads(f.read().decode("utf-8"))  
      file_id = data['id']

    # 현재 읽고 있는 파일에 포함된 문서를 하나씩 처리하기
    for document in data['document']:
      # 발화 목록 업데이트:
      for utterance in document['utterance']:
        utterance.update({'speaker_id': f'{file_id}_{utterance["speaker_id"]}'})
        utterances_list.append(utterance)

      # 발화자 목록 업데이트:
      # 문서의 메타 정보에서 발화자 값을 하나씩 처리하기
      for speaker in document['metadata']['speaker']:
        speaker.update({'speaker_id': f'{file_id}_{speaker["id"]}'})
        speakers_list.append(speaker)

100%|██████████| 47424/47424 [00:18<00:00, 2508.57it/s]


In [38]:
print(utterances_list[0])

{'id': 'MDRW2100000001.1.1', 'form': '안녕하세요', 'original_form': '안녕하세요', 'speaker_id': 'MDRW2100000001_1', 'time': '20210512 16:04'}


In [39]:
print(speakers_list[0])

{'id': '1', 'age': '20대', 'occupation': '전문가 및 관련 종사자', 'sex': '여성', 'birthplace': '울산', 'principal_residence': '전북', 'current_residence': '경기', 'device': '스마트폰', 'keyboard': '2벌식(쿼티)', 'speaker_id': 'MDRW2100000001_1'}


In [42]:
df_utterances = pd.DataFrame(utterances_list).to_csv('../NIKL_OM_utterances.csv')
df_speakers = pd.DataFrame(speakers_list).drop_duplicates().to_csv('../NIKL_OM_speakers.csv')

In [43]:
!tar -czf ../NIKL_OM_utterances.csv.tar.gz ../NIKL_OM_utterances.csv
!tar -czf ../NIKL_OM_speakers.csv.tar.gz ../NIKL_OM_speakers.csv

In [44]:
!rm ../NIKL_OM_2021_v1.1.zip
!rm ../NIKL_OM_speakers.csv
!rm ../NIKL_OM_utterances.csv